In [ ]:
# !pip uninstall -y scikit-learn
# !pip install scikit-learn==1.3.1

In [ ]:
# ! pip install --upgrade pip
# ! pip install --user xgboost seaborn
# ! pip install --user bayesian-optimization

In [ ]:
# import mplhep
import sys

import seaborn as sns

import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.colors as colors

from sklearn.datasets import make_classification,make_regression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc,roc_curve,confusion_matrix,classification_report,precision_recall_curve,mean_squared_error,accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate, validation_curve,train_test_split,KFold,learning_curve,cross_val_score
from scipy.stats import ks_2samp

import xgboost
from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import randint, uniform

In [ ]:
plt.rcParams.update({
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 18
})

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/07-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal","All","BB","ccbar","ddbar","ssbar","taupair","uubar","uds"]
GenEvents = ["Signal","BB","ccbar","ddbar","ssbar","taupair","uubar"]

DataFrames = {}  # define empty dictionary to hold dataframes

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/TopoAna/Completed_TopoAna/TopoAna_"+ s +".root:Dstree",library='pd')

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

The line below is to look at the available variables.

In [ ]:
DataFrames["All"].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
# DataFrames[samples[0]] = DataFrames[samples[0]][DataFrames[samples[0]]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames[samples[1]] = DataFrames[samples[1]][DataFrames[samples[1]]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["ccbar"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
# DataFrames[samples[0]] = DataFrames[samples[0]][DataFrames[samples[0]]['Ds_chiProb_rank']==1]
# DataFrames[samples[1]] = DataFrames[samples[1]][DataFrames[samples[1]]['Ds_chiProb_rank']==1]

# D0 Invariant Mass
#-----------------------
# DataFrames[samples[0]] = DataFrames[samples[0]][(DataFrames[samples[0]]['Ds_D0_sideband']==1)]
# DataFrames[samples[1]] = DataFrames[samples[1]][(DataFrames[samples[1]]['Ds_D0_sideband']==1)]

# $D^{*+}$ Plots

In [ ]:
# === Settings ===
Stacked = True
Density = False
Bins = 50
Range = [0.0, 0.25]
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# Cut range on 'Ds_diff_D0pi'
cut_low = 0.142
cut_high = 0.15

# Variable to plot after cut
var = 'Ds_massDifference_0'

# Labels and colors
labels = [
    r'$c \bar{c}$',
    r'$u \bar{u}, \; d \bar{d}, \;s \bar{s}$',
    r'$BB$',
    r'$\tau^{+} \tau^{-}$'
]

# Apply sideband cut (outside signal region) and collect data
data = [
    DataFrames["ccbar"].query("Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high")[var],
    DataFrames["uds"].query("Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high")[var],
    DataFrames["BB"].query("Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high")[var],
    DataFrames["taupair"].query("Ds_diff_D0pi <= @cut_low or Ds_diff_D0pi >= @cut_high")[var],
]

# === Plot ===
# plt.figure(figsize=(8, 5))
plt.hist(data[::-1],
         label=labels[::-1],
         density=Density,
         stacked=Stacked,
         bins=Bins,
         range=Range,
         histtype='step',
         linewidth=2)

# Titles
plt.title(r'$D_s^{+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] e^{+} \nu_{e}$' + '\n' + r'$\Delta m_{\pi}(D_s^{+} - D^{0}) \notin [0.142,\; 0.15] \; \mathrm{GeV}/c^{2}$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")

# Labels
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
plt.legend()
# plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# === Settings ===
Stacked = True
Density = False
Bins = 50
Range = [0.0, 0.25]
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# Data source and variables
df_cut = DataFrames["All"]
cut_var = "Ds_diff_D0pi"
plot_var = 'Ds_massDifference_0'
pdg_var = 'Ds_mcPDG'

# === Categories based on true Ds_mcPDG ===
dstar_plus = df_cut[abs(df_cut[pdg_var]) == 413][plot_var]
dstar_zero = df_cut[abs(df_cut[pdg_var]) == 423][plot_var]
other = df_cut[(abs(df_cut[pdg_var]) != 413) & (abs(df_cut[pdg_var]) != 423)][plot_var]

# === Plot ===
plt.hist([other, dstar_zero, dstar_plus],
         color=["#2E2E2E", "#4C6EB1", "#007C91"],
         label=["Other", r"$D^{*0}$", r"$D^{*+}$"],
         density=Density,
         stacked=Stacked,
         bins=Bins,
         range=Range,
         histtype='step',
         linewidth=2)

# Titles and labels
plt.title(r'$D_s^{+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] e^{+} \nu_{e}$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# === Settings ===
Stacked = True
Density = False
Bins = 50
Range = [0.0, 0.25]
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# Data source and variables
df = DataFrames["All"]
cut_var = "Ds_diff_D0pi"
plot_var = 'Ds_massDifference_0'
pdg_var = 'Ds_mcPDG'

# Sideband cut (exclude D*⁺ peak)
cut_low = 0.142
cut_high = 0.15
df_cut = df.query(f"{cut_var} <= @cut_low or {cut_var} >= @cut_high")

# === Categories based on true Ds_mcPDG ===
dstar_plus = df_cut[abs(df_cut[pdg_var]) == 413][plot_var]
dstar_zero = df_cut[abs(df_cut[pdg_var]) == 423][plot_var]
other = df_cut[(abs(df_cut[pdg_var]) != 413) & (abs(df_cut[pdg_var]) != 423)][plot_var]

# === Plot ===
plt.hist([other, dstar_zero, dstar_plus],
         color=["#2E2E2E", "#4C6EB1", "#007C91"],
         label=["Other", r"$D^{*0}$", r"$D^{*+}$"],
         density=Density,
         stacked=Stacked,
         bins=Bins,
         range=Range,
         histtype='step',
         linewidth=2)

# Titles and labels
plt.title(r'$D_s^{+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] e^{+} \nu_{e}$' + '\n' +
          r'$\Delta m_{\pi}(D_s^{+} - D^{0}) \notin [0.142,\; 0.15] \; \mathrm{GeV}/c^{2}$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def plot_2d_with_marginals(df, xvar, yvar, xrange=None, yrange=None, bins=50, title=None):
    # Extract variables
    x = df[xvar]
    y = df[yvar]

    # Apply range cuts if specified
    if xrange is not None:
        xmask = (x >= xrange[0]) & (x <= xrange[1])
    else:
        xmask = pd.Series([True] * len(x))

    if yrange is not None:
        ymask = (y >= yrange[0]) & (y <= yrange[1])
    else:
        ymask = pd.Series([True] * len(y))

    # Combined mask
    mask = xmask & ymask
    x = x[mask]
    y = y[mask]

    # Set up figure layout
    fig = plt.figure(figsize=(6, 6))
    gs = gridspec.GridSpec(4, 4, hspace=0.05, wspace=0.05)

    ax_main = fig.add_subplot(gs[1:4, 0:3])
    ax_xhist = fig.add_subplot(gs[0, 0:3], sharex=ax_main)
    ax_yhist = fig.add_subplot(gs[1:4, 3], sharey=ax_main)
    ax_cbar = fig.add_subplot(gs[0, 3])

    # Define histogram range
    hist_range = None
    if xrange is not None and yrange is not None:
        hist_range = [xrange, yrange]

    # 2D histogram
    # counts, xedges, yedges, im = ax_main.hist2d(x, y, bins=bins, range=hist_range, cmap="viridis", norm=colors.LogNorm())  # <-- This is the key)
    counts, xedges, yedges, im = ax_main.hist2d(x, y, bins=bins, range=hist_range, cmap="viridis")
    cbar = fig.colorbar(im, cax=ax_cbar)
    cbar.set_label("Entries")

    # Marginal histograms
    ax_xhist.hist(x, bins=bins, range=xrange, color="#2E2E2E", histtype='step', linewidth=1.5)
    ax_yhist.hist(y, bins=bins, range=yrange, orientation="horizontal", color="#007C91", histtype='step', linewidth=1.5)

    # Axis labels
    ax_main.set_xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
    ax_main.set_ylabel(r'$\Delta m_{\pi}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
    ax_xhist.set_ylabel("Entries")
    ax_yhist.set_xlabel("Entries")

    # Clean ticks
    plt.setp(ax_xhist.get_xticklabels(), visible=False)
    plt.setp(ax_yhist.get_yticklabels(), visible=False)

    # Optional title
    if title:
        plt.suptitle(title, fontsize=14)

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space for title
    plt.show()

In [ ]:
plot_2d_with_marginals(DataFrames["All"],
    xvar="Ds_massDifference_0",
    yvar="Ds_diff_D0pi",
    xrange=(0.0, 0.25),
    yrange=(0.1, 0.55),
    bins=60,
    title="Generic Events"
)

In [ ]:
plot_2d_with_marginals(DataFrames["Signal"],
    xvar="Ds_massDifference_0",
    yvar="Ds_diff_D0pi",
    xrange=(0.0, 0.25),
    yrange=(0.1, 0.60),
    bins=60,
    title="Signal"
)

## $D^{*+}$ BDT Cut

In [ ]:
cut_low = 0.142
cut_high = 0.151

# Apply sideband cut to 'All' sample
DataFrames["All"] = DataFrames["All"][
    (DataFrames["All"]["Ds_diff_D0pi"] <= cut_low) | (DataFrames["All"]["Ds_diff_D0pi"] >= cut_high)
]

# Apply sideband cut to each generator-level sample
for s in GenEvents:
    DataFrames[s] = DataFrames[s][
        (DataFrames[s]["Ds_diff_D0pi"] <= cut_low) | (DataFrames[s]["Ds_diff_D0pi"] >= cut_high)
    ]

# Fake $D^0$ Suppression

In [ ]:
DataFrames["All"].isna().sum()

In [ ]:
DataFrames["All"]["D0_isSignal"] = DataFrames["All"]["D0_isSignal"].replace(np.nan, 0)

In [ ]:
DataFrames["All"]["Ds_isSignal"] = DataFrames["All"]["Ds_isSignal"].replace(np.nan, 0)

for s in GenEvents[0:]: # loop over samples
    DataFrames[s]["Ds_isSignal"] = DataFrames[s]["Ds_isSignal"].replace(np.nan, 0)

In [ ]:
Variables = [
            #  'K_dr',
             'pi_dr',
            #  'K_kaonID',
            #  'pi_pionID',
             'D0_dM',
             'D0_chiProb',
             'D0_flightDistance',
             'D0_useCMSFrame_p',
             'D0_cos_decayAngle_1',
            #  'D0_daughterAngle_0_1',
             ]

features = [
            #  r'$dr(K^{-})$',
             r'$dr(\pi^{+})$',
            #  r'$kaonID(K^{-})$',
            #  r'$pionID(\pi^{+})$',
             r'$m(D^{0}) - m_{PDG}(D^{0})$',
             r'$p-value(D^{0})$',
             r'$Flight \; Distance(D^{0})$',
             r'$p^{*} (D^{0})$',
             r'$\cos\theta^*_{daughter_{1}}$',
            #  r'$\Delta \theta (K^{-} \pi^{+})$',
             ]

In [ ]:
plt.figure(figsize=(18, 15))

heatmap = sns.heatmap(DataFrames["Signal"][Variables].corr(), annot=True, cmap="coolwarm",vmin=-1, vmax=1)

heatmap.set_title('Signal Correlation Heatmap', fontdict={'fontsize':20}, pad=16)

In [ ]:
plt.figure(figsize=(18, 15))

heatmap = sns.heatmap(DataFrames["All"][Variables].corr(), annot=True, cmap="coolwarm",vmin=-1, vmax=1)

heatmap.set_title('Background Correlation Heatmap', fontdict={'fontsize':20}, pad=16)

## Data Preprocessing

In [ ]:
# Define your features and labels from the 'All' dataset
X = DataFrames["All"][Variables].to_numpy(dtype=np.float32)
y = DataFrames["All"]['D0_isSignal'].to_numpy(dtype=np.int64)

# # Reference variable for decorrelation — this is what uBoost will try to flatten for background
# ref_variable = DataFrames["All"]["D0_dM"]

In [ ]:
#splitting with  Holdout method for eval_set
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30,
                                                    random_state=42,
                                                    # stratify=y
                                                    )

## Model Training

In [ ]:
# # Compute the positive class weight
# pos_class_weight = (len(y) - np.sum(y)) / np.sum(y)

# Calculate class imbalance ratio
neg, pos = np.bincount(y)  # 0s and 1s
scale = neg / pos

# Create EarlyStopping callback
early_stop = xgboost.callback.EarlyStopping(
    rounds=10,
    metric_name='rmse',
    data_name="validation_0",
    save_best=True,
)

In [ ]:
eval_set = [(X_test, y_test)]
xgbm_final = XGBClassifier(objective="binary:logistic",
                    eval_metric="logloss",
                    # early_stopping_rounds=10,
                    # scale_pos_weight=pos_class_weight,
                    scale_pos_weight=scale,
                    max_delta_step=1,
                    random_state=42,
                    n_estimators=100)

xgbm_final.fit(X_train, y_train, 
        eval_set=[(X_train, y_train),(X_test, y_test)], 
        # early_stopping_rounds=5,
        verbose=0) 

In [ ]:
# loss curve of xgboost
results = xgbm_final.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["logloss"], label="Training loss")
plt.plot(results["validation_1"]["logloss"], label="Validation loss")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

## Parameter Optimization
This optimization is pulling too much resources and ending the connection

In [ ]:
# param_dist = {
#     "learning_rate": uniform(0.01, 0.2),        # e.g., 0.01 to 0.21
#     "max_depth": randint(1, 5),                 # 1 to 4
#     "n_estimators": randint(100, 201),          # 100 to 200
#     "reg_lambda": randint(1, 5),
#     "gamma": randint(0, 4),
#     "subsample": uniform(0.5, 0.5),             # 0.5 to 1.0
#     "min_child_weight": randint(1, 6),
#     "colsample_bytree": uniform(0.3, 0.7)
# }

# random_search = RandomizedSearchCV(
#     bdt,
#     param_distributions=param_dist,
#     n_iter=50,  # Try only 50 random combos (you can adjust)
#     cv=5,
#     n_jobs=-1,
#     random_state=42,
#     verbose=1
# )

# # After running RandomizedSearchCV:
# random_search.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)

# # Extract best parameters and apply them to the base model
# xgbm_final = bdt.set_params(**random_search.best_params_, random_state=17).fit(X_train, y_train)

# cv_results = cross_validate(xgbm_final, X, y, cv=10,
#                             scoring=["f1"],return_train_score=True)

# print(cv_results['train_f1'].mean())
# print(cv_results['test_f1'].mean())

In [ ]:
# print("Best parameters found by RandomizedSearchCV:")
# for param, value in random_search.best_params_.items():
#     print(f"{param:20s}: {value}")

## Feature Importance

In [ ]:
# Get feature importance scores
print(xgbm_final.feature_importances_)

In [ ]:
def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features})
    plt.figure(figsize=(16, 8))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

In [ ]:
plot_importance(xgbm_final, features)
# When the feature importance graph is observed, 
# it is seen that the variables other than a02 and a01 are important for the xgboost model.

## Overfitting Check

In [ ]:
from scipy import stats
def get_pulls(counts,errors,pdf):
    pull = (-pdf + counts) / errors
    return pull

In [ ]:
def compare_train_test(clf, X_train, y_train, X_test, y_test):
    Density = True
    decisions = [] # list to hold decisions of classifier
    for X,y in ((X_train, y_train), (X_test, y_test)): # train and test
        if hasattr(clf, "predict_proba"): # if predict_proba function exists
            d1 = clf.predict_proba(X[y<0.5])[:, 1] # background
            d2 = clf.predict_proba(X[y>0.5])[:, 1] # signal
        else: # predict_proba function doesn't exist
            X_tensor = torch.as_tensor(X, dtype=torch.float) # make tensor from X_test_scaled
            y_tensor = torch.as_tensor(y, dtype=torch.long) # make tensor from y_test
            X_var, y_var = Variable(X_tensor), Variable(y_tensor) # make variables from tensors
            d1 = clf(X_var[y_var<0.5])[1][:, 1].cpu().detach().numpy() # background
            d2 = clf(X_var[y_var>0.5])[1][:, 1].cpu().detach().numpy() # signal
        decisions += [d1, d2] # add to list of classifier decision

    #pd.set_option('max_columns', None)
#     %config InlineBackend.figure_format = 'retina'
    # plt.style.use('belle2')
    lw=3

    fig,axs=plt.subplots(3,1,figsize=(10,10),gridspec_kw={'height_ratios':[1,0.2,0.2]})

    bins = 50
    bin_edges = np.linspace(0,1,bins)
    
    test_bkg_count_weight=bins/len(decisions[2])
    test_sig_count_weight=bins/len(decisions[3])
    test_bkg_counts,test_bkg_bins = np.histogram(decisions[2],bins=bins,range=(0,1))
    test_sig_counts,test_sig_bins = np.histogram(decisions[3],bins=bins,range=(0,1))

    train_bkg_counts,train_bkg_bins,_etc=axs[0].hist(decisions[0],color = 'tab:blue',
            histtype='step',bins=bins,density=Density,range=(0,1),linewidth=lw,label='Train Background')
    train_sig_counts,train_sig_bins,_etc=axs[0].hist(decisions[1],color = 'tab:red',
            histtype='step',bins=bins,density=Density,range=(0,1),linewidth=lw,label=r'Train Signal')
    axs[0].hist(decisions[0],color = 'tab:blue',
            histtype='stepfilled',alpha=0.4,bins=bins,density=Density,range=(0,1))
    axs[0].hist(decisions[1],color = 'tab:red',
            histtype='stepfilled',alpha=0.4,bins=bins,density=Density,range=(0,1))
    bin_width=test_bkg_bins[1]-test_bkg_bins[0]
    bin_centers=[el+(bin_width/2) for el in test_bkg_bins[:-1]]

    axs[0].errorbar(bin_centers,test_bkg_count_weight*test_bkg_counts,
                yerr=test_bkg_count_weight*np.sqrt(test_bkg_counts),label='Test Background',color='tab:blue',
                marker='o',linewidth=lw,ls='')
    axs[0].errorbar(bin_centers,test_sig_count_weight*test_sig_counts,
                yerr=test_sig_count_weight*np.sqrt(test_sig_counts),label='Test Signal',color='tab:red',
                marker='o',linewidth=lw,ls='')
    axs[0].set_title(r'$D_{s}^{+} \rightarrow D^{0} e^{+} \nu_{e}$',loc='left')
    axs[0].set_xlim(0,1)
    axs[0].set_ylim(0)
    axs[0].set_ylabel('Event Density')

    x= decisions[1]
    y=  decisions[3]
    ks_p_value_sig = ks_2samp(x, y)[1]

    x= decisions[0]
    y= decisions[2]
    ks_p_value_bkg = ks_2samp(x, y)[1]

    leg=axs[0].legend(loc='upper center',title=f"Sig K-S test score: {ks_p_value_sig:0.3f}"+
                      "\n"+f"Bkg K-S test score: {ks_p_value_bkg:0.3f}")
    leg._legend_box.align = "left"  

    pulls=get_pulls(test_bkg_count_weight*test_bkg_counts,test_bkg_count_weight*np.sqrt(test_bkg_counts),np.array(train_bkg_counts))
    axs[1].bar(bin_centers,pulls,width=bin_width)
    axs[1].set_xlim(0,1)
    axs[1].set_ylabel('Pulls')
    axs[1].set_ylim(-5,5)

    pulls=get_pulls(test_sig_count_weight*test_sig_counts,test_sig_count_weight*np.sqrt(test_sig_counts),np.array(train_sig_counts))
    axs[2].bar(bin_centers,pulls,width=bin_width,color='tab:red')
    axs[2].set_xlim(0,1)
    axs[2].set_ylabel('Pulls')
    axs[2].set_ylim(-5,5)
    axs[2].set_xlabel(r'BDT output')

    return decisions

In [ ]:
decisions = compare_train_test(xgbm_final, X_train, y_train, X_test, y_test)

## Model Check

### Basf2 ROC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y_score_test = xgbm_final.predict_proba(X_test)[:, 1]
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_score_test)
area_test = auc(fpr_test, tpr_test)

y_score_train = xgbm_final.predict_proba(X_train)[:, 1]
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_score_train)
area_train = auc(fpr_train, tpr_train)

# Get classifier scores (probabilities for class 1)
train_scores = xgbm_final.predict_proba(X_train)[:, 1]
test_scores  = xgbm_final.predict_proba(X_test)[:, 1]

# Use y_train and y_test to separate signal/background
sig_train = train_scores[y_train == 1]
bkg_train = train_scores[y_train == 0]
sig_test  = test_scores[y_test == 1]
bkg_test  = test_scores[y_test == 0]

# Optionally, group them into one list like this:
decisions = [bkg_train, sig_train, bkg_test, sig_test]

bdt_cuts = np.linspace(0, 1, 100)

sig_eff_train = []
bkg_rej_train = []
sig_eff_test = []
bkg_rej_test = []
fom_vals = []

for cut in bdt_cuts:
    num_sig_train = np.sum(sig_train > cut)
    num_bkg_train = np.sum(bkg_train > cut)
    num_sig_test = np.sum(sig_test > cut)
    num_bkg_test = np.sum(bkg_test > cut)

    # FoM calculation
    fom = num_sig_test / np.sqrt(num_sig_test + num_bkg_test) if (num_sig_test + num_bkg_test) > 0 else 0
    fom_vals.append(fom)

    sig_eff_train.append(num_sig_train / len(sig_train))
    bkg_rej_train.append(1 - (num_bkg_train / len(bkg_train)))
    sig_eff_test.append(num_sig_test / len(sig_test))
    bkg_rej_test.append(1 - (num_bkg_test / len(bkg_test)))

# Find optimal FoM point
fom_vals = np.array(fom_vals)
best_idx = np.argmax(fom_vals)
best_cut = bdt_cuts[best_idx]

# Plot
fig, axs = plt.subplots(1, 1, figsize=(7, 6))
lw = 2

# axs.plot([0, 1], [0, 1], color='grey', linestyle='--', label='Random')
axs.plot(bkg_rej_train, sig_eff_train, color='tab:blue', linewidth=lw, label=f'Train (AUC = {area_train:.2f})')
axs.plot(bkg_rej_test, sig_eff_test, color='tab:red', linestyle='--', linewidth=lw, label=f'Test (AUC = {area_test:.2f})')

# ① Shade the overfit gap
axs.fill_between(bkg_rej_test,
                 sig_eff_train,
                 sig_eff_test,
                 where=(np.array(sig_eff_train) > np.array(sig_eff_test)),
                 color='gray', alpha=0.2, label='Overfit Gap')

# ② Mark the optimal cut point (from test curve)
axs.axhline(sig_eff_test[best_idx], color='black', ls='--', linewidth=1.6,
            label=f'Best FoM Cut = {best_cut:.3f}')
axs.axvline(bkg_rej_test[best_idx], color='black', ls='--', linewidth=1.6)
axs.scatter(bkg_rej_test[best_idx], sig_eff_test[best_idx], color='green', s=50)

# Axis labels and formatting
axs.set_title(r'$D_{s}^{+} \rightarrow D^{0} e^{+} \nu_{e}$', loc='left')
axs.set_ylim(0, 1.05)
axs.set_xlim(0, 1.05)
axs.set_xlabel('Background rejection')
axs.set_ylabel('Signal efficiency')
axs.legend(loc='lower left')
axs.grid(True)
plt.tight_layout()
plt.show()

### Machine Learing ROC

In [ ]:
y_score_test = xgbm_final.predict_proba(X_test)[:, 1]
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_score_test)
area_test = auc(fpr_test, tpr_test)

y_score_train = xgbm_final.predict_proba(X_train)[:, 1]
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_score_train)
area_train = auc(fpr_train, tpr_train)

plt.plot([0, 1], [0, 1], color='grey', linestyle='--')
plt.plot(fpr_test, tpr_test, label=f'Test ROC curve (AUC = {area_test:.2f})')
plt.plot(fpr_train, tpr_train, label=f'Train ROC curve (AUC = {area_train:.2f})')
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
# We can make the plot look nicer by forcing the grid to be square
plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
# Make predictions on the test set
y_pred_proba = xgbm_final.predict_proba(X_test)[:, 1]

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"ROC AUC Score: {roc_auc:.2f}")

### Other Checks

In [ ]:
# Predict on training and validation sets
train_preds = xgbm_final.predict(X_train)
val_preds = xgbm_final.predict(X_test)

# Calculate accuracy scores
train_accuracy = accuracy_score(y_train, train_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Check for large difference between train and validation accuracy
if train_accuracy - val_accuracy > 0.1:
    print("Warning: The model may be overfitting!")

Check if XGBoost Is Underfitting

In [ ]:
# Predict on training and validation sets
train_preds = xgbm_final.predict(X_train)
val_preds = xgbm_final.predict(X_test)

# Calculate MSE for training and validation sets
train_mse = mean_squared_error(y_train, train_preds)
val_mse = mean_squared_error(y_test, val_preds)

print(f"Training MSE: {train_mse:.4f}")
print(f"Validation MSE: {val_mse:.4f}")

# Check if both training and validation MSE are high
if train_mse > 100 and val_mse > 100:
    print("Warning: The model may be underfitting!")
    print("Consider increasing model complexity by adding more estimators, reducing learning rate, or adjusting other hyperparameters.")

## BDT Cut Optimization

In [ ]:
DataFrames["All"]["Ds_FakeD0BDT"] = xgbm_final.predict_proba(DataFrames["All"][Variables])[:,1]

for s in GenEvents[0:]: # loop over samples
    DataFrames[s]["Ds_FakeD0BDT"] = xgbm_final.predict_proba(DataFrames[s][Variables])[:,1]

In [ ]:
def compute_fom_curve(scores, labels, weights=None, n_thresholds=200):
    """
    Compute FoM (S / sqrt(S + B)) across multiple BDT score thresholds.
    
    Parameters:
        scores (np.array): BDT scores for the validation/test set
        labels (np.array): True labels (1 for real D0, 0 for fake)
        weights (np.array): Optional per-event weights
        n_thresholds (int): Number of thresholds to scan (default=200)

    Returns:
        thresholds (np.array), foms (np.array), best_threshold (float), best_fom (float)
    """
    thresholds = np.linspace(0, 1, n_thresholds)
    foms = []

    for t in thresholds:
        mask = scores > t

        if weights is not None:
            S = np.sum(weights[(labels == 1) & mask])
            B = np.sum(weights[(labels == 0) & mask])
        else:
            S = np.sum((labels == 1) & mask)
            B = np.sum((labels == 0) & mask)

        fom = S / np.sqrt(S + B) if (S + B) > 0 else 0
        foms.append(fom)

    foms = np.array(foms)
    best_idx = np.argmax(foms)
    return thresholds, foms, thresholds[best_idx], foms[best_idx]

In [ ]:
# Predict scores from your trained model
scores = xgbm_final.predict_proba(X_test)[:, 1]

# Optionally define weights (or leave as None)
weights = np.ones_like(y_test)  # or from your MC truth if applicable

# Compute FoM curve
thresholds, foms, best_thresh, best_fom = compute_fom_curve(scores, y_test)

# Print results
print(f"Best threshold: {best_thresh:.3f}")
print(f"Best FoM: {best_fom:.3f}")

# Plot it
plt.plot(thresholds, foms)
plt.axvline(best_thresh, color='red', linestyle='--', label=f'Best = {best_thresh:.3f}')
plt.axvspan(0,best_thresh,color='gray',alpha=0.2)
plt.xlabel("BDT Threshold")
plt.ylabel("FoM = S / √(S + B)")
plt.title("FoM Scan vs BDT Threshold")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# from Functions import get_df_sig_bkg,get_nsig_nbkg,plot_var_sig_bkg,optimize_SB,plot_save
# # "Ds_extraInfo_BkgBDT"
# # "Ds_BS"
# optimize_SB(df_sig=DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal']==1)],
#             df_bkg=DataFrames["All"],
#             var="Ds_FakeD0BDT",
#             Signal=DataFrames["Signal"],
#             Background=DataFrames["All"],
#             FoM="Ds_FakeD0BDT",
#             xlabel='Classifier Output',
#             Bins=50,Range=[0,1],
#             varmin=0,varmax=0.99,select='right',Width=False)

## Fake $D^0$ BDT Cut

In [ ]:
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]["Ds_FakeD0BDT"]>=0.182)]

# for s in GenEvents[0:]: # loop over samples
#     DataFrames[s] = DataFrames[s][(DataFrames[s]["Ds_FakeD0BDT"]>=0.182)]

# Background Suppression

In [ ]:
DataFrames[samples[0]].isna().sum()

In [ ]:
print("Signal Number: ",len(DataFrames[samples[0]]))
print("Background Number: ",len(DataFrames[samples[1]]))

## Variable Comparison

In [ ]:
plt.style.use('default')
plt.rcParams.update({
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 16
})

In [ ]:
Stacked = False
Density = True
Bins = 50
Range = [0, 0.4]
Op = -1
dM = -1
# i = "Ds_gammaveto_M_Correction"
i = 'Ds_Ds_starminusDs_M_Correction'
# i = 'Ds_extraInfo_FakeD0BDT'
# i = 'Ds_chiProb'
perBin = ((Range[1] - Range[0])/Bins)*1000
# perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Signal$'
label2= r'$Background$'

labels=[label1,label2]
colors=["#1f77b4","#d62728"]

data = [
        DataFrames["Signal"][i], # (DataFrames["Signal"]['Ds_charge']==-1) & 
        DataFrames["All"][i]
       ]


plt.hist(data, color=colors, label=labels, alpha=1, range=Range, stacked=Stacked, density=Density, linewidth=2, bins=Bins, histtype='step')
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.58,ls='--',color='gray')

# Title
#---------
# plt.title(r'$Signal: Charge(D_s^{+})=Positive$', loc = "left")
# plt.title(r'$\bf Signal\;Events$', loc = "right")
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# plt.xlabel(r'$Fake \; D^{0} \; Suppression(D^{0})$')
# plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# plt.yscale("log") 
# plt.xscale("log") 
plt.legend()
plt.show()

## Variable Correlation

In [ ]:
Variables = [
            # 'Ds_massDifference_0',
             "Ds_FakeD0BDT",
            #  'K_kaonID','K_abs_dr',
            #  'pi_pionID','pi_abs_dr',
            #  'D0_cos_decayAngle_1',
            #  'D0_chiProb','D0_flightDistance','D0_useCMSFrame_p','D0_M'
            #  'e_pt',
            #  'Ds_chiProb_noIP',
            'Ds_chiProb',
            #  'Ds_KaonCount_3',
            'Ds_gammaveto_M_Correction',
            'Ds_Ds_starminusDs_M_Correction',
            #  'e_pt',
            #  'e_cos_theta',
            #  'e_d0','e_z0'
            # 'K_dr','pi_dr',
            # 'K_kaonID','pi_pionID',
            # 'D0_dM','D0_chiProb',
            # 'D0_flightDistance',
            # 'D0_useCMSFrame_p',
            # 'D0_cos_decayAngle_1',
            # #  'D0_daughterAngle_0_1',
             ]

features = [
            # 'Ds_massDifference_0',
            r'$Fake D^{0} Suppresion$',
            # 'e_pt',
            # r'$\Delta m(D_s^{*+} - D_{s}^{+})$',
            # r'$p-value(D_{s}^{+})$',
            r'$p-value_{IP}(D_{s}^{+})$',
            # r'$n_{K^{-}} - n_{K^{+}} + n_{K_{s}^{0}} (ROE)$',
            r'$m(e_{sig}^{+} e_{ROE}^{-})$',
            r'$\Delta m(D_s^{*+} - D_{s}^{+})$',
            # r'$p_{t} (e^{+})$',
            # 'e_cos_theta',
            # 'e_d0','e_z0',
            # 'K_dr','pi_dr',
            # 'K_kaonID','pi_pionID',
            # 'D0_dM','D0_chiProb',
            # 'D0_flightDistance',
            # 'D0_useCMSFrame_p',
            # 'D0_cos_decayAngle_1',
            # #  'D0_daughterAngle_0_1',
            ]

In [ ]:
plt.figure(figsize=(18, 15))

heatmap = sns.heatmap(DataFrames["Signal"][Variables].corr(), annot=True, cmap="coolwarm",vmin=-1, vmax=1)

heatmap.set_title('Signal Correlation Heatmap', fontdict={'fontsize':20}, pad=16)

In [ ]:
plt.figure(figsize=(18, 15))

heatmap = sns.heatmap(DataFrames["All"][Variables].corr(), annot=True, cmap="coolwarm",vmin=-1, vmax=1)

heatmap.set_title('Background Correlation Heatmap', fontdict={'fontsize':20}, pad=16)

## Data Preprocessing

In [ ]:
#  Organise data ready for the machine learning model

# for sklearn data are usually organised
# into one 2D array of shape (n_samples x n_features)
# containing all the data and one array of categories
# of length n_samples

all_MC = []  # define empty list that will contain all features for the MC
for s in GenEvents:  # loop over the different samples
    if s != "data":  # only MC should pass this
        all_MC.append(
            DataFrames[s][Variables]
        )  # append the MC dataframe to the list containing all MC features
X = np.concatenate(
    all_MC
)  # concatenate the list of MC dataframes into a single 2D array of features, called X

all_y = (
    []
)  # define empty list that will contain labels whether an event in signal or background
for s in GenEvents:  # loop over the different samples
    if s != "data":  # only MC should pass this
        if "Signal" in s:  # only signal MC should pass this
            all_y.append(
                np.ones(DataFrames[s].shape[0], dtype=np.int32)
            )  # signal events are labelled with 1
        else:  # only background MC should pass this
            all_y.append(
                np.zeros(DataFrames[s].shape[0], dtype=np.int32)
            )  # background events are labelled 0
y = np.concatenate(
    all_y
)  # concatenate the list of labels into a single 1D array of labels, called y

In [ ]:
#splitting with  Holdout method for eval_set
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30,
                                                    random_state=42,
                                                    # stratify=y
                                                    )

## Model Training

In [ ]:
# # Compute the positive class weight
# pos_class_weight = (len(y) - np.sum(y)) / np.sum(y)

# Calculate class imbalance ratio
neg, pos = np.bincount(y)  # 0s and 1s
scale = neg / pos

# Create EarlyStopping callback
early_stop = xgboost.callback.EarlyStopping(
    rounds=10,
    metric_name='rmse',
    data_name="validation_0",
    save_best=True,
)

In [ ]:
eval_set = [(X_test, y_test)]
xgbm_final = XGBClassifier(objective="binary:logistic",
                    eval_metric="logloss",
                    # early_stopping_rounds=10,
                    scale_pos_weight=scale,
                    max_delta_step=1,
                    random_state=42,
                    n_estimators=100)

xgbm_final.fit(X_train, y_train, 
        eval_set=[(X_train, y_train),(X_test, y_test)], 
        # early_stopping_rounds=5,
        verbose=0) 

In [ ]:
# loss curve of xgboost
results = xgbm_final.evals_result()

plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["logloss"], label="Training loss")
plt.plot(results["validation_1"]["logloss"], label="Validation loss")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

## Parameter Optimization

This optimization is pulling too much resources and ending the connection

In [ ]:
# param_dist = {
#     "learning_rate": uniform(0.01, 0.2),        # e.g., 0.01 to 0.21
#     "max_depth": randint(1, 5),                 # 1 to 4
#     "n_estimators": randint(100, 201),          # 100 to 200
#     "reg_lambda": randint(1, 5),
#     "gamma": randint(0, 4),
#     "subsample": uniform(0.5, 0.5),             # 0.5 to 1.0
#     "min_child_weight": randint(1, 6),
#     "colsample_bytree": uniform(0.3, 0.7)
# }

# random_search = RandomizedSearchCV(
#     bdt,
#     param_distributions=param_dist,
#     n_iter=50,  # Try only 50 random combos (you can adjust)
#     cv=5,
#     n_jobs=-1,
#     random_state=42,
#     verbose=1
# )

# # After running RandomizedSearchCV:
# random_search.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)

# # Extract best parameters and apply them to the base model
# xgbm_final = bdt.set_params(**random_search.best_params_, random_state=17).fit(X_train, y_train)

# cv_results = cross_validate(xgbm_final, X, y, cv=10,
#                             scoring=["f1"],return_train_score=True)

# print(cv_results['train_f1'].mean())
# print(cv_results['test_f1'].mean())

In [ ]:
# print("Best parameters found by RandomizedSearchCV:")
# for param, value in random_search.best_params_.items():
#     print(f"{param:20s}: {value}")

## Feature Importance

In [ ]:
# Get feature importance scores
print(xgbm_final.feature_importances_)

In [ ]:
def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features})
    plt.figure(figsize=(16, 8))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

In [ ]:
plot_importance(xgbm_final, features)
# When the feature importance graph is observed, 
# it is seen that the variables other than a02 and a01 are important for the xgboost model.

## Overfitting Check

In [ ]:
from scipy import stats
def get_pulls(counts,errors,pdf):
    pull = (-pdf + counts) / errors
    return pull

In [ ]:
def compare_train_test(clf, X_train, y_train, X_test, y_test):
    decisions = [] # list to hold decisions of classifier
    for X,y in ((X_train, y_train), (X_test, y_test)): # train and test
        if hasattr(clf, "predict_proba"): # if predict_proba function exists
            d1 = clf.predict_proba(X[y<0.5])[:, 1] # background
            d2 = clf.predict_proba(X[y>0.5])[:, 1] # signal
        else: # predict_proba function doesn't exist
            X_tensor = torch.as_tensor(X, dtype=torch.float) # make tensor from X_test_scaled
            y_tensor = torch.as_tensor(y, dtype=torch.long) # make tensor from y_test
            X_var, y_var = Variable(X_tensor), Variable(y_tensor) # make variables from tensors
            d1 = clf(X_var[y_var<0.5])[1][:, 1].cpu().detach().numpy() # background
            d2 = clf(X_var[y_var>0.5])[1][:, 1].cpu().detach().numpy() # signal
        decisions += [d1, d2] # add to list of classifier decision

    #pd.set_option('max_columns', None)
#     %config InlineBackend.figure_format = 'retina'
    # plt.style.use('belle2')
    lw=3

    fig,axs=plt.subplots(3,1,figsize=(10,10),gridspec_kw={'height_ratios':[1,0.2,0.2]})

    bins = 50
    bin_edges = np.linspace(0,1,bins)
    
    test_bkg_count_weight=bins/len(decisions[2])
    test_sig_count_weight=bins/len(decisions[3])
    test_bkg_counts,test_bkg_bins = np.histogram(decisions[2],bins=bins,range=(0,1))
    test_sig_counts,test_sig_bins = np.histogram(decisions[3],bins=bins,range=(0,1))

    train_bkg_counts,train_bkg_bins,_etc=axs[0].hist(decisions[0],color = 'tab:blue',
            histtype='step',bins=bins,density=True,range=(0,1),linewidth=lw,label='Train Background')
    train_sig_counts,train_sig_bins,_etc=axs[0].hist(decisions[1],color = 'tab:red',
            histtype='step',bins=bins,density=True,range=(0,1),linewidth=lw,label=r'Train Signal')
    axs[0].hist(decisions[0],color = 'tab:blue',
            histtype='stepfilled',alpha=0.4,bins=bins,density=True,range=(0,1))
    axs[0].hist(decisions[1],color = 'tab:red',
            histtype='stepfilled',alpha=0.4,bins=bins,density=True,range=(0,1))
    bin_width=test_bkg_bins[1]-test_bkg_bins[0]
    bin_centers=[el+(bin_width/2) for el in test_bkg_bins[:-1]]

    axs[0].errorbar(bin_centers,test_bkg_count_weight*test_bkg_counts,
                yerr=test_bkg_count_weight*np.sqrt(test_bkg_counts),label='Test Background',color='tab:blue',
                marker='o',linewidth=lw,ls='')
    axs[0].errorbar(bin_centers,test_sig_count_weight*test_sig_counts,
                yerr=test_sig_count_weight*np.sqrt(test_sig_counts),label='Test Signal',color='tab:red',
                marker='o',linewidth=lw,ls='')
    axs[0].set_title(r'$D_{s}^{+} \rightarrow D^{0} e^{+} \nu_{e}$',loc='left')
    axs[0].set_xlim(0,1)
    axs[0].set_ylim(0)
    axs[0].set_ylabel('Event Density')

    x= decisions[1]
    y=  decisions[3]
    ks_p_value_sig = ks_2samp(x, y)[1]

    x= decisions[0]
    y= decisions[2]
    ks_p_value_bkg = ks_2samp(x, y)[1]

    leg=axs[0].legend(loc='upper center',title=f"Sig K-S test score: {ks_p_value_sig:0.3f}"+
                      "\n"+f"Bkg K-S test score: {ks_p_value_bkg:0.3f}")
    leg._legend_box.align = "left"  

    pulls=get_pulls(test_bkg_count_weight*test_bkg_counts,test_bkg_count_weight*np.sqrt(test_bkg_counts),np.array(train_bkg_counts))
    axs[1].bar(bin_centers,pulls,width=bin_width)
    axs[1].set_xlim(0,1)
    axs[1].set_ylabel('Pulls')
    axs[1].set_ylim(-5,5)

    pulls=get_pulls(test_sig_count_weight*test_sig_counts,test_sig_count_weight*np.sqrt(test_sig_counts),np.array(train_sig_counts))
    axs[2].bar(bin_centers,pulls,width=bin_width,color='tab:red')
    axs[2].set_xlim(0,1)
    axs[2].set_ylabel('Pulls')
    axs[2].set_ylim(-5,5)
    axs[2].set_xlabel(r'BDT output')

    return decisions

In [ ]:
decisions = compare_train_test(xgbm_final, X_train, y_train, X_test, y_test)

## Model Check

### Basf2 ROC

In [ ]:
# # compute ROC 
# sig_train=decisions[1]
# sig_test=decisions[3]
# bkg_train=decisions[0]
# bkg_test=decisions[2]

# bdt_cuts=np.linspace(0,1,100)
# sig_efficiency_train=[]
# bkg_rejection_train=[]
# den_sig_train=len(sig_train)
# den_bkg_train=len(bkg_train)

# sig_efficiency_test=[]
# bkg_rejection_test=[]
# den_sig_test=len(sig_test)
# den_bkg_test=len(bkg_test)


# for cut in bdt_cuts:
#     num_sig_train=len([el for el in sig_train if el>cut])
#     num_bkg_train=len([el for el in bkg_train if el>cut])
#     num_sig_test=len([el for el in sig_test if el>cut])
#     num_bkg_test=len([el for el in bkg_test if el>cut])
    
#     sig_efficiency_test.append(num_sig_test/den_sig_test)
#     bkg_rejection_test.append(1-(num_bkg_test/den_bkg_test))
#     sig_efficiency_train.append(num_sig_train/den_sig_train)
#     bkg_rejection_train.append(1-(num_bkg_train/den_bkg_train))

# fig,axs=plt.subplots(1,1,figsize=(8,6))
# lw=2
# axs.plot([1, 0], [0, 1], color='grey', linestyle='--')
# axs.plot(bkg_rejection_train,sig_efficiency_train,color='tab:blue',marker='',linewidth=lw,label='Train')
# axs.plot(bkg_rejection_test,sig_efficiency_test,color='tab:red',marker='',linewidth=lw,ls='--',label='Test')
# axs.set_title(r'$D_{s}^{+} \rightarrow D^{0} e^{+} \nu_{e}$',loc='left')

# axs.set_ylim(0,1.05)
# axs.set_xlim(0,1.05)
# axs.legend(loc='lower left')
# axs.set_xlabel('Background rejection')
# axs.set_ylabel('Signal efficiency')
# plt.tight_layout()

# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y_score_test = xgbm_final.predict_proba(X_test)[:, 1]
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_score_test)
area_test = auc(fpr_test, tpr_test)

y_score_train = xgbm_final.predict_proba(X_train)[:, 1]
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_score_train)
area_train = auc(fpr_train, tpr_train)

# Get classifier scores (probabilities for class 1)
train_scores = xgbm_final.predict_proba(X_train)[:, 1]
test_scores  = xgbm_final.predict_proba(X_test)[:, 1]

# Use y_train and y_test to separate signal/background
sig_train = train_scores[y_train == 1]
bkg_train = train_scores[y_train == 0]
sig_test  = test_scores[y_test == 1]
bkg_test  = test_scores[y_test == 0]

# Optionally, group them into one list like this:
decisions = [bkg_train, sig_train, bkg_test, sig_test]

bdt_cuts = np.linspace(0, 1, 100)

sig_eff_train = []
bkg_rej_train = []
sig_eff_test = []
bkg_rej_test = []
fom_vals = []

for cut in bdt_cuts:
    num_sig_train = np.sum(sig_train > cut)
    num_bkg_train = np.sum(bkg_train > cut)
    num_sig_test = np.sum(sig_test > cut)
    num_bkg_test = np.sum(bkg_test > cut)

    # FoM calculation
    fom = num_sig_test / np.sqrt(num_sig_test + num_bkg_test) if (num_sig_test + num_bkg_test) > 0 else 0
    fom_vals.append(fom)

    sig_eff_train.append(num_sig_train / len(sig_train))
    bkg_rej_train.append(1 - (num_bkg_train / len(bkg_train)))
    sig_eff_test.append(num_sig_test / len(sig_test))
    bkg_rej_test.append(1 - (num_bkg_test / len(bkg_test)))

# Find optimal FoM point
fom_vals = np.array(fom_vals)
best_idx = np.argmax(fom_vals)
best_cut = bdt_cuts[best_idx]

# Plot
fig, axs = plt.subplots(1, 1, figsize=(7, 6))
lw = 2

# axs.plot([0, 1], [0, 1], color='grey', linestyle='--', label='Random')
axs.plot(bkg_rej_train, sig_eff_train, color='tab:blue', linewidth=lw, label=f'Train (AUC = {area_train:.2f})')
axs.plot(bkg_rej_test, sig_eff_test, color='tab:red', linestyle='--', linewidth=lw, label=f'Test (AUC = {area_test:.2f})')

# ① Shade the overfit gap
axs.fill_between(bkg_rej_test,
                 sig_eff_train,
                 sig_eff_test,
                 where=(np.array(sig_eff_train) > np.array(sig_eff_test)),
                 color='gray', alpha=0.2, label='Overfit Gap')

# ② Mark the optimal cut point (from test curve)
axs.axhline(sig_eff_test[best_idx], color='black', ls='--', linewidth=1.6,
            label=f'Best FoM Cut = {best_cut:.3f}')
axs.axvline(bkg_rej_test[best_idx], color='black', ls='--', linewidth=1.6)
axs.scatter(bkg_rej_test[best_idx], sig_eff_test[best_idx], color='green', s=50)

# Axis labels and formatting
axs.set_title(r'$D_{s}^{+} \rightarrow D^{0} e^{+} \nu_{e}$', loc='left')
axs.set_ylim(0, 1.05)
axs.set_xlim(0, 1.05)
axs.set_xlabel('Background rejection')
axs.set_ylabel('Signal efficiency')
axs.legend(loc='lower left')
axs.grid(True)
plt.tight_layout()
plt.show()


### Machine Learing ROC

In [ ]:
y_score_test = xgbm_final.predict_proba(X_test)[:, 1]
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_score_test)
area_test = auc(fpr_test, tpr_test)

y_score_train = xgbm_final.predict_proba(X_train)[:, 1]
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_score_train)
area_train = auc(fpr_train, tpr_train)

plt.plot([0, 1], [0, 1], color='grey', linestyle='--')
plt.plot(fpr_test, tpr_test, label=f'Test ROC curve (AUC = {area_test:.2f})')
plt.plot(fpr_train, tpr_train, label=f'Train ROC curve (AUC = {area_train:.2f})')
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.0)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
# We can make the plot look nicer by forcing the grid to be square
plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
# Make predictions on the test set
y_pred_proba = xgbm_final.predict_proba(X_test)[:, 1]

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"ROC AUC Score: {roc_auc:.2f}")

### Other Checks

Check if XGBoost Is Overfitting

In [ ]:
# Predict on training and validation sets
train_preds = xgbm_final.predict(X_train)
val_preds = xgbm_final.predict(X_test)

# Calculate accuracy scores
train_accuracy = accuracy_score(y_train, train_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Check for large difference between train and validation accuracy
if train_accuracy - val_accuracy > 0.1:
    print("Warning: The model may be overfitting!")

Check if XGBoost Is Underfitting

In [ ]:
# Predict on training and validation sets
train_preds = xgbm_final.predict(X_train)
val_preds = xgbm_final.predict(X_test)

# Calculate MSE for training and validation sets
train_mse = mean_squared_error(y_train, train_preds)
val_mse = mean_squared_error(y_test, val_preds)

print(f"Training MSE: {train_mse:.4f}")
print(f"Validation MSE: {val_mse:.4f}")

# Check if both training and validation MSE are high
if train_mse > 100 and val_mse > 100:
    print("Warning: The model may be underfitting!")
    print("Consider increasing model complexity by adding more estimators, reducing learning rate, or adjusting other hyperparameters.")

## BDT Cut Optimization

In [ ]:
def compute_fom_curve(scores, labels, weights=None, n_thresholds=200):
    """
    Compute FoM (S / sqrt(S + B)) across multiple BDT score thresholds.
    
    Parameters:
        scores (np.array): BDT scores for the validation/test set
        labels (np.array): True labels (1 for real D0, 0 for fake)
        weights (np.array): Optional per-event weights
        n_thresholds (int): Number of thresholds to scan (default=200)

    Returns:
        thresholds (np.array), foms (np.array), best_threshold (float), best_fom (float)
    """
    thresholds = np.linspace(0, 1, n_thresholds)
    foms = []

    for t in thresholds:
        mask = scores > t

        if weights is not None:
            S = np.sum(weights[(labels == 1) & mask])
            B = np.sum(weights[(labels == 0) & mask])
        else:
            S = np.sum((labels == 1) & mask)
            B = np.sum((labels == 0) & mask)

        fom = S / np.sqrt(S + B) if (S + B) > 0 else 0
        foms.append(fom)

    foms = np.array(foms)
    best_idx = np.argmax(foms)
    return thresholds, foms, thresholds[best_idx], foms[best_idx]

In [ ]:
# Predict scores from your trained model
scores = xgbm_final.predict_proba(X_test)[:, 1]

# Optionally define weights (or leave as None)
weights = np.ones_like(y_test)  # or from your MC truth if applicable

# Compute FoM curve
thresholds, foms, best_thresh, best_fom = compute_fom_curve(scores, y_test)

# Print results
print(f"Best threshold: {best_thresh:.3f}")
print(f"Best FoM: {best_fom:.3f}")

# Plot it
plt.plot(thresholds, foms)
plt.axvline(best_thresh, color='red', linestyle='--', label=f'Best = {best_thresh:.3f}')
plt.axvspan(0,best_thresh,color='gray',alpha=0.2)
plt.xlabel("BDT Threshold")
plt.ylabel("FoM = S / √(S + B)")
plt.title("FoM Scan vs BDT Threshold")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
DataFrames["Signal"]["Ds_BkgBDT"] = xgbm_final.predict_proba(DataFrames["Signal"][Variables])[:,1]
DataFrames["All"]["Ds_BkgBDT"] = xgbm_final.predict_proba(DataFrames["All"][Variables])[:,1]

In [ ]:
# from Functions import get_df_sig_bkg,get_nsig_nbkg,plot_var_sig_bkg,optimize_SB,plot_save

# optimize_SB(df_sig=DataFrames["Signal"][(abs(DataFrames["Signal"]['Ds_isSignal'])==1)],
#             df_bkg=DataFrames["All"],
#             var="Ds_extraInfo_BkgBDT",
#             Signal=DataFrames["Signal"][(abs(DataFrames["Signal"]['Ds_isSignal'])==1)],
#             Background=DataFrames["All"],
#             FoM="Ds_extraInfo_BkgBDT",
#             xlabel='Classifier Output',
#             Bins=50,Range=[0,1],
#             varmin=0,varmax=0.93,select='right',Width=False)

# Plots

In [ ]:
plt.style.use('default')
plt.rcParams.update({
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 16
})

In [ ]:
print(DataFrames["All"].query('Ds_BkgBDT>=0.657')[['D0_isSignal']].value_counts(normalize=False,dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'D0_dM'
Range = [-0.02, 0.02]
BD = 0.657
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$Other$'
label1= r'$Other \; (FDS \; BDT \geq 0.8)$'

labels=[label1]
colors = ['C5']
data=[
      DataFrames["All"][((abs(DataFrames["All"]['Ds_D0_Other'])==1) | ((abs(DataFrames["All"]['D0_mcPDG'])==421) & (abs(DataFrames["All"]['D0_isSignal'])==0))) & (DataFrames["All"]["Ds_FakeD0BDT"]>=BD)][var],
      ]

plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(0.02,0.04,color='gray',alpha=0.2)
# plt.axvline(0.02,ls='--',color='gray')
# plt.axvline(0.04,ls='--',color='gray')

# Title
#--------
plt.title(r'$\bf Generic \; Events$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$m(D^{0}) - m_{PDG}(D^{0}) \;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 500)
plt.legend()
plt.show()

Suggested Background Break-up

In [ ]:
# Stacked = False
# Density = False
# Bins = 50
# Range = [0.1, 0.6]
# Op = 0.721
# dM = -1
# # i = 'e_cos_theta'
# # i = 'Ds_vpho_CMS_daughterAngle'
# i = 'Ds_diff_D0pi'
# # i = 'Ds_chiProb_noIP'
# # i = 'Ds_chiProb'
# # i = 'Ds_extraInfo_FastBDT'
# # i = 'D0_chiProb'
# # i = 'Ds_Ds_starminusDs_M_Correction'
# # i = "Ds_gammaveto_M_Correction"
# # i = 'D0_chiProb'
# # i = "Ds_L_diff"
# # var = 'e_cos_theta'
# # i = 'e_pt'
# perBin = ((Range[1] - Range[0])/Bins)*1000
# # perBin = ((Range[1] - Range[0])/Bins)
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
# label3= r'$D^{0}$'
# label4= r'$Other$'

# labels1=[label1,label3,label4]
# colors1=['C1','C2','C3']
# data1=[
#       DataFrames["ccbar"][(abs(DataFrames["ccbar"]['Ds_D0_Dstarplus'])==1) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["ccbar"]["Ds_BkgBDT"]>=Op)][i],
#       DataFrames["ccbar"][(abs(DataFrames["ccbar"]['Ds_D0_NoDstarplusDstar0'])==1) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["ccbar"]["Ds_BkgBDT"]>=Op)][i],
#       DataFrames["ccbar"][(abs(DataFrames["ccbar"]['Ds_D0_Other'])==1) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["ccbar"]["Ds_BkgBDT"]>=Op)][i],
#       ]
# labels2=[r'$D^{*0} \; (Comb.)$',r'$D^{*0} \; (Peak)$']
# colors2=['C4','C5']
# data2=[
#       DataFrames["ccbar"][(abs(DataFrames["ccbar"]['Ds_mcPDG'])!=423) & (abs(DataFrames["ccbar"]['Ds_D0_Dstar0'])==1) & (DataFrames["ccbar"]["Ds_BkgBDT"]>=Op)][i],
#       DataFrames["ccbar"][(abs(DataFrames["ccbar"]['Ds_mcPDG'])==423) & (abs(DataFrames["ccbar"]['Ds_D0_Dstar0'])==1) & (DataFrames["ccbar"]["Ds_BkgBDT"]>=Op)][i],
#       ]

# # factor = 0.1
# # plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_BS"]>=Op)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_BS"]>=Op)][i]), ls='--', linewidth=1.5)
# plt.hist(data1, color=colors1, label=labels1, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.hist(data2, color=colors2, label=labels2, density=Density, stacked=False, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# # plt.axvline(0.58,ls='--',color='gray')

# # Title
# #--------
# plt.title(r'$BDT \; \geq 0.721$', loc = "left")
# # Label
# #-------
# # plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# # plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width = perBin))
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# # plt.xlabel(r'$p_{t} (e^{+}) [GeV/c]$')
# # plt.xlabel(r'$\Delta \theta(D_s^{+} \; K^{+/-}/K_{s}^{0}) \; [rad]$')
# # plt.xlabel(r'$cos\theta \; (e^{+})$')
# # plt.xlabel(r'$p-value(D^{0})$')
# # plt.xlabel(r'$p-value(D_{s}^{+})$')
# # plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# # plt.xlabel(r'$Fake D^{0} Suppression(D^{0})$')
# # plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$p_{t} \; (e^{+})\;[GeV/c]$')
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# # plt.xlabel(r'$cos\theta \; (e^{+})$')
# # plt.xlabel(r'$p-value(D^{0})$')
# # plt.xlabel(r'$\mid \vec{x}_{D_{s}^{+}} - \vec{x}_{D^{0}} \mid \; [cm]$')
# # plt.xlabel(r'$dz \; (e^{+}) \; [cm]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 500)
# plt.legend()
# plt.show()

In [ ]:
Stacked = True
Density = False
Bins = 50
# var = 'Ds_diff_D0pi'
var = 'Ds_massDifference_0'
Range = [0.0, 0.25]
BS = 0.566
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$Prompt \; D^{0}$'
label3= r'$D^{*0} \rightarrow D^{0} X$'
label4= r'$D^{*+} \rightarrow D^{0} X$'

labels=[label1,label2,label3,label4]
colors=['C5','C4','C1','C2',]
data=[
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]["Ds_BkgBDT"]>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]["Ds_BkgBDT"]>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]["Ds_BkgBDT"]>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]["Ds_BkgBDT"]>=BS)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]["Ds_BkgBDT"]>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]["Ds_BkgBDT"]>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$\bf Generic \; Events$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1443.999$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 30000)
plt.legend()
plt.show()

In [ ]:
Stacked = True
Density = False
Bins = 50
# i = 'Ds_diff_D0pi'
i = 'Ds_massDifference_0'
Range = [0.0, 0.25]
BS = 0.5
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Comb.$'
label2= r'$NaN$'
label3= r'$D^{*0}$'
label4= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'

labels=[label1,label2,label3,label4]
colors=["#DD8452","#C44E52","#55A868","#4C72B0"]
data=[
      DataFrames["All"][((abs(DataFrames["All"]["Ds_mcPDG"])!=413) & (abs(DataFrames["All"]["Ds_mcPDG"])!=423) & (~DataFrames["All"]["Ds_mcPDG"].isna())) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]["Ds_BkgBDT"]>=BS)][i],
      DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna()) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]["Ds_BkgBDT"]>=BS)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==423) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]["Ds_BkgBDT"]>=BS)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==413) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]["Ds_BkgBDT"]>=BS)][i]
      ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$e^{+}$ mass hypothesis: pion', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1443.999$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 30000)
plt.legend()
plt.show()

In [ ]:
Bins=50
Density = False
Stacked = True
Range = [0.1,0.55]
BS = 0.566
perBin = ((Range[1] - Range[0])/Bins)*1000
var = 'Ds_diff_D0pi'
# var = 'Ds_massDifference_0'
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$isSignal(D_s^{+})=1$'
label2= r'$isSignal(D_s^{+})=0$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=['#7eb0d5','#fd7f6f','purple']

data = [DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal']==1) & (DataFrames["Signal"]["Ds_BkgBDT"]>=BS)][var],
        DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal']==0) & (DataFrames["Signal"]["Ds_BkgBDT"]>=BS)][var],
        DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'].isna()) & (DataFrames["Signal"]["Ds_BkgBDT"]>=BS)][var]
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, linewidth=1.5, stacked=Stacked, density=Density, bins=Bins, histtype='step')
plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
plt.axvline(0.16,ls='--',color='gray')

# Title
#---------
# Signal
plt.title(r'$2M\;Events$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# # Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log") 
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# === Settings ===
Stacked = True
Density = False
Bins = 50
Range = [0.0, 0.25]
BDT = 0.6
perBin = ((Range[1] - Range[0]) / Bins) * 1000
print("Width Per Bin: {:.2f} MeV".format(perBin))

# Data source and variables
df = DataFrames["All"][(DataFrames["All"]["Ds_BkgBDT"]>=BDT)]
cut_var = "Ds_diff_D0pi"
plot_var = 'Ds_massDifference_0'
pdg_var = 'Ds_mcPDG'

# Sideband cut (exclude D*⁺ peak)
cut_low = 0.142
cut_high = 0.15
df_cut = df.query(f"{cut_var} <= @cut_low or {cut_var} >= @cut_high")

# === Categories based on true Ds_mcPDG ===
dstar_plus = df_cut[abs(df_cut[pdg_var]) == 413][plot_var]
dstar_zero = df_cut[abs(df_cut[pdg_var]) == 423][plot_var]
other = df_cut[(abs(df_cut[pdg_var]) != 413) & (abs(df_cut[pdg_var]) != 423)][plot_var]

# === Plot ===
plt.hist([other, dstar_zero, dstar_plus],
         color=["#2E2E2E", "#4C6EB1", "#007C91"],
         label=["Other", r"$D^{*0}$", r"$D^{*+}$"],
         density=Density,
         stacked=Stacked,
         bins=Bins,
         range=Range,
         histtype='step',
         linewidth=2)

# Titles and labels
plt.title(r'$D_s^{+} \rightarrow [D^{0} \rightarrow K^{-} \pi^{+}] e^{+} \nu_{e}$' + '\n' +
          r'$\Delta m_{\pi}(D_s^{+} - D^{0}) \notin [0.142,\; 0.15] \; \mathrm{GeV}/c^{2}$', loc="left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc="right")
plt.xlabel(r'$\Delta m_{e}(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
plt.legend()
plt.tight_layout()
plt.show()

# Save BDT Output

In [ ]:
# import uproot
# import os

# # Make sure the output directory exists
# output_dir = "/group/belle2/users2022/amubarak/02-Grid/ML_Trained/"
# os.makedirs(output_dir, exist_ok=True)

# # Now save each DataFrame into its own ROOT file
# for s in samples:
#     out_path = os.path.join(output_dir, f"TopoAna_{s}_withBDT.root")
#     with uproot.recreate(out_path) as f:
#         f["Dstree"] = DataFrames[s]  # Save the DataFrame into a tree named 'Dstree'